In [13]:
#import libraries necessary
import pymysql
from sqlalchemy import create_engine, text
import pandas as pd
from configparser import ConfigParser

#set up a parser object and file path to parse a text file in order to protect username and password for MySQL
parser = ConfigParser()
configPath = r'C:\Users\melac\Desktop\Practice\Jupyter Notebooks\ConfigFileSQLChicagoCensus.txt'
_ = parser.read(configPath)
parser.sections()

conn = create_engine(parser.get('my_db', 'conn'))

## Chicago Cenus Data Analysis

This notebook analyzes three socioeconomic data sets taken from the Chicago Data Portal. Use of both Python and SQL languages are incorporated. The data is stored in a MySQL server, and the pandas library is used to communicate with the server. Any data with missing values has been excluded. Sources and details on the data sets used can be found below:

[Socioeconomic Indicators in Chicago](https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2)

[Chicago Public Schools](https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t)

[Chicago Crime Data](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2)

### Problem 1.1

##### Write and execute a SQL query to list the school names, community names and average attendance for communities with a hardship index of 98.

In [15]:
#The following code was executed in the MySQL database in order to create the view being called.
#CREATE OR REPLACE VIEW community_school_attendance AS (
#    SELECT schools.NAME_OF_SCHOOL, schools.COMMUNITY_AREA_NAME, schools.AVERAGE_STUDENT_ATTENDANCE
#    FROM chicagopublicschools schools, chicagocensusdata census
#    WHERE HARDSHIP_INDEX=98 AND schools.COMMUNITY_AREA_NAME = census.COMMUNITY_AREA_NAME
#    ORDER BY AVERAGE_STUDENT_ATTENDANCE DESC);
query = text("SELECT * FROM community_school_attendance")
df = pd.read_sql_query(query, conn)
display(df)

,NAME_OF_SCHOOL,COMMUNITY_AREA_NAME,AVERAGE_STUDENT_ATTENDANCE
0,William E B Dubois Elementary School,RIVERDALE,93.30%
1,Ira F Aldridge Elementary School,RIVERDALE,92.90%
2,George Washington Carver Military Academy High...,RIVERDALE,91.60%
3,George Washington Carver Primary School,RIVERDALE,90.90%


### Problem 1.2

##### Write and execute a SQL query to list all crimes that took place at a school. Include case number, crime type and community name.

In [16]:
##The following code was executed in the MySQL database in order to create the view being called.
#CREATE OR REPLACE VIEW crimes_in_schools AS (
#    SELECT crime.CASE_NUMBER, crime.PRIMARY_TYPE, census.COMMUNITY_AREA_NAME
#    FROM chicagocrimedata crime, chicagocensusdata census
#    WHERE LOCATION_DESCRIPTION LIKE "%SCHOOL%" AND crime.COMMUNITY_AREA_NUMBER = census.COMMUNITY_AREA_NUMBER
#    ORDER BY PRIMARY_TYPE ASC);
query = ("SELECT * FROM crimes_in_schools")
df = pd.read_sql_query(query, conn)
display(df)

,CASE_NUMBER,PRIMARY_TYPE,COMMUNITY_AREA_NAME
0,HT315369,ASSAULT,East Garfield Park
1,HL725506,BATTERY,Lincoln Square
2,HH639427,BATTERY,Austin
3,HP716225,BATTERY,Douglas
4,HL353697,BATTERY,South Shore
5,JA460432,BATTERY,Ashburn
6,HS200939,CRIMINAL DAMAGE,Austin
7,HR585012,CRIMINAL TRESPASS,Ashburn
8,HK577020,NARCOTICS,Rogers Park
9,HS305355,NARCOTICS,Brighton Park


### Problem 2.1

##### Write and execute a SQL statement to create a view showing the columns listed in the following table, with new column names as shown in the second column.
|<b>Column name in CHICAGO_PUBLIC_SCHOOLS</b>|<b>Column name in view</b>|
|---|---|
|NAME_OF_SCHOOL|School_Name|
|Safety_Icon|Safety_Rating|
|Family_Involvement|Family_Rating|
|Environment_Icon|Environment_Rating|
|Instruction_Icon|Instruction_Rating|
|Leaders_Icon|Leaders_Rating|
|Teachers_Icon|Teachers_Rating|

##### - Write and execute a SQL statement that returns all of the columns from the view.
##### - Write and execute a SQL statement that returns just the school name and leaders rating from the view.

In [18]:
##The following code was executed in the MySQL database in order to create the view being called.
#CREATE OR REPLACE VIEW chicago_public_schools AS (
#    SELECT NAME_OF_SCHOOL AS School_Name, Safety_Icon AS Safety_Rating, 
#    Family_Involvement_Score AS Family_Rating, Environment_Icon AS Environment_Rating, 
#    Instruction_Icon AS Instruction_Rating, Leaders_Icon AS Leaders_Rating, 
#    Teachers_Icon AS Teachers_Rating
#    FROM chicagopublicschools);
query = "SELECT * FROM chicago_public_schools LIMIT 5"
df = pd.read_sql_query(query, conn)
display(df)

,School_Name,Safety_Rating,Family_Rating,Environment_Rating,Instruction_Rating,Leaders_Rating,Teachers_Rating
0,Abraham Lincoln Elementary School,Very Strong,99,Strong,Strong,Weak,Strong
1,Adam Clayton Powell Paideia Community Academy ...,Average,66,Strong,Very Strong,Weak,Strong
2,Adlai E Stevenson Elementary School,Strong,NDA,Average,Weak,Weak,NDA
3,Agustin Lara Elementary Academy,Average,44,Average,Weak,Weak,Average
4,Air Force Academy High School,Average,60,Strong,Average,Weak,Average


In [19]:
#A limit was placed on the results to save space in the notebook once rendered
query = "SELECT School_Name, Leaders_Rating FROM chicago_public_schools LIMIT 5"
df = pd.read_sql_query(query, conn)
display(df)

,School_Name,Leaders_Rating
0,Abraham Lincoln Elementary School,Weak
1,Adam Clayton Powell Paideia Community Academy ...,Weak
2,Adlai E Stevenson Elementary School,Weak
3,Agustin Lara Elementary Academy,Weak
4,Air Force Academy High School,Weak


### Problem 3.1

##### Write the structure of a query to create or replace a stored procedure called UPDATE_LEADERS_SCORE that takes a in_School_ID parameter as an integer and a in_Leader_Score parameter as an integer.

In [ ]:
#The structure of a stored procedure in MySQL is as follows:
#DELIMITER //
#CREATE PROCEDURE UPDATE_LEADERS_SCORE (IN in_School_ID INT, IN in_Leader_Score VARCHAR(20))
#BEGIN
#    Statements;
#END //

### Problem 3.2

##### Inside your stored procedure, write a SQL statement to update the Leaders_Score field in the CHICAGO_PUBLIC_SCHOOLS table for the school identified by in_School_ID to the value in the in_Leader_Score parameter.

In [ ]:
#The SQL completed SQL Procedure:
#DELIMITER //
#CREATE PROCEDURE UPDATE_LEADERS_SCORE (IN in_School_ID INT, IN in_Leader_Score varchar(15))
#BEGIN
#    UPDATE chicagopublicschools 
#    SET Leaders_Score = in_Leader_Score
#    WHERE School_ID = in_School_ID;
#END //

### Problem 3.3

##### Inside your stored procedure, write a SQL IF statement to update the Leaders_Icon field in the CHICAGO_PUBLIC_SCHOOLS table for the school identified by in_School_ID using the following information.
|<b>Score Lower Limit</b>|<b>Score Upper Limit</b>|<b>Icon</b>|
|---|---|---|
|80|90|Very Strong|
|60|79|Strong|
|40|59|Average|
|20|39|Weak|
|0|19|Very Weak|


In [ ]:
#The procedure, including the IF statements, are below:
#DELIMITER //
#CREATE PROCEDURE UPDATE_LEADERS_SCORE (IN in_School_ID INT, IN in_Leader_Score varchar(15))
#BEGIN
#    UPDATE chicagopublicschools schools
#    SET Leaders_Score = in_Leader_Score
#    WHERE School_ID = in_School_ID;
#    
#    IF in_Leader_Score >= 80 THEN
#        UPDATE chicagopublicschools
#        SET Leaders_Icon = "Very Strong"
#        WHERE School_ID = in_School_ID;
#    ELSEIF in_Leader_Score >= 60 THEN
#        UPDATE chicagopublicschools
#        SET Leaders_Icon = "Strong"
#        WHERE School_ID = in_School_ID;
#    ELSEIF in_Leader_Score >= 40 THEN
#        UPDATE chicagopublicschools
#        SET Leaders_Icon = "AVERAGE"
#        WHERE School_ID = in_School_ID;
#    ELSEIF in_Leader_Score >= 20 THEN
#        UPDATE chicagopublicschools
#        SET Leaders_Icon = "Weak"
#        WHERE School_ID = in_School_ID;
#    ELSE
#        UPDATE chicagopublicschools
#        SET Leaders_Icon = "Very Weak"
#        WHERE School_ID = in_School_ID;
#    END IF;   
#END //

### Problem 3.4

##### Run your code to create the stored procedure. Write a query to call the stored procedure, passing a valid school ID and a leader score of 50, to check that the procedure works as expected.

In [28]:
#The code to call the function in MySQL is below
#CALL UPDATE_LEADERS_SCORE(610038, 50)
query = "SELECT School_ID, Leaders_Score, Leaders_Icon FROM chicagopublicschools WHERE School_ID = 610038;"
df = pd.read_sql_query(query, conn)
display(df)

,School_ID,Leaders_Score,Leaders_Icon
0,610038,50,AVERAGE


### Problem 4.1

##### Update your stored procedure definition. Add a generic ELSE clause to the IF statement that rolls back the current work if the score did not fit any of the preceding categories.

In [ ]:
#DELIMITER //
#CREATE PROCEDURE UPDATE_LEADERS_SCORE (IN in_School_ID INT, IN in_Leader_Score varchar(15))
#BEGIN
#    IF in_Leader_Score >= 80 AND in_Leader_Score <= 90 THEN
#        UPDATE chicagopublicschools
#        SET Leaders_Score = in_Leader_Score, Leaders_Icon = "Very Strong"
#        WHERE School_ID = in_School_ID;
#    ELSEIF in_Leader_Score >= 60 AND in_Leader_Score <= 79 THEN
#        UPDATE chicagopublicschools
#        SET Leaders_Score = in_Leader_Score, Leaders_Icon = "Strong"
#        WHERE School_ID = in_School_ID;
#    ELSEIF in_Leader_Score >= 40 AND in_Leader_Score <= 59 THEN
#        UPDATE chicagopublicschools
#        SET Leaders_Score = in_Leader_Score, Leaders_Icon = "AVERAGE"
#        WHERE School_ID = in_School_ID;
#    ELSEIF in_Leader_Score >= 20 AND in_Leader_Score <= 39 THEN
#        UPDATE chicagopublicschools
#        SET Leaders_Score = in_Leader_Score, Leaders_Icon = "Weak"
#        WHERE School_ID = in_School_ID;
#    ELSEIF in_Leader_Score >= 0 AND in_Leader_Score <= 19 THEN
#        UPDATE chicagopublicschools
#        SET Leaders_Score = in_Leader_Score, Leaders_Icon = "Very Weak"
#        WHERE School_ID = in_School_ID;
#    ELSE 
#        ROLLBACK;
#    END IF;
#    END //

### Problem 4.2

##### Update your stored procedure definition again. Add a statement to commit the current unit of work at the end of the procedure. Run your code to replace the stored procedure. 
##### - Write and run one query to check that the updated stored procedure works as expected when you use a valid score of 38.
##### - Write and run another query to check that the updated stored procedure works as expected when you use an invalid score of 101.

In [ ]:
#The code below is the final procedure
#SET AUTOCOMMIT=false;

#DELIMITER //
#CREATE PROCEDURE UPDATE_LEADERS_SCORE (IN in_School_ID INT, IN in_Leader_Score varchar(15))
#BEGIN
#    IF in_Leader_Score >= 80 AND in_Leader_Score <= 90 THEN
#        UPDATE chicagopublicschools
#        SET Leaders_Score = in_Leader_Score, Leaders_Icon = "Very Strong"
#        WHERE School_ID = in_School_ID;
#    ELSEIF in_Leader_Score >= 60 AND in_Leader_Score <= 79 THEN
#        UPDATE chicagopublicschools
#        SET Leaders_Score = in_Leader_Score, Leaders_Icon = "Strong"
#        WHERE School_ID = in_School_ID;
#    ELSEIF in_Leader_Score >= 40 AND in_Leader_Score <= 59 THEN
#        UPDATE chicagopublicschools
#        SET Leaders_Score = in_Leader_Score, Leaders_Icon = "AVERAGE"
#       WHERE School_ID = in_School_ID;
#    ELSEIF in_Leader_Score >= 20 AND in_Leader_Score <= 39 THEN
#        UPDATE chicagopublicschools
#        SET Leaders_Score = in_Leader_Score, Leaders_Icon = "Weak"
#        WHERE School_ID = in_School_ID;
#    ELSEIF in_Leader_Score >= 0 AND in_Leader_Score <= 19 THEN
#        UPDATE chicagopublicschools
#        SET Leaders_Score = in_Leader_Score, Leaders_Icon = "Very Weak"
#        WHERE School_ID = in_School_ID;
#    ELSE 
#        ROLLBACK;
#    END IF;
#   
#    COMMIT;
#    END //

In [29]:
#This code block is the call to the procedure to set the score to 38
#followed by the query to display the results
#CALL UPDATE_LEADERS_SCORE(610038, 38)
query = "SELECT School_ID, Leaders_Icon, Leaders_Score FROM chicagopublicschools WHERE School_ID = 610038"
df = pd.read_sql_query(query, conn)
display(df)

,School_ID,Leaders_Icon,Leaders_Score
0,610038,Weak,38


In [30]:
#This code block is the call to the procedure to set the score to 101
#followed by the query to display the results
#CALL UPDATE_LEADERS_SCORE(610038, 101)
query = "SELECT School_ID, Leaders_Icon, Leaders_Score FROM chicagopublicschools WHERE School_ID = 610038"
df = pd.read_sql_query(query, conn)
display(df)

,School_ID,Leaders_Icon,Leaders_Score
0,610038,Weak,38
